In [6]:
import torch
from torchvision import transforms
import timm #  модуль для использования предварительно обученных моделей
from PIL import Image
import numpy as np
import os
from tqdm import tqdm # для отображения прогресса выполнения кода
from timeit import default_timer as timer

In [7]:
# создаем объект для преобразования изображения
_transform=transforms.Compose([
                       transforms.Resize((224,224)),  # изменяем размер изображения до 224x224
                       transforms.ToTensor(),  # преобразуем изображение в тензор
                       transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])  # нормализуем значения пикселей

# функция для чтения изображения
def read_img_file(f):
    img = Image.open(f)  # открываем изображение
    img=img.convert('RGB')  # преобразуем в формат RGB
    return img

In [8]:
# список моделей для тестирования
models = ["tf_efficientnetv2_m_in21k", "vit_base_patch16_224_in21k", "resnetv2_50x1_bitm_in21k"]

# открываем файл с классами изображений
with open("./imagenet21k_wordnet_lemmas.txt") as f:
    classes = [line.strip() for line in f.readlines()]

# получаем список папок с изображениями для тестирования
folders = os.listdir("./dataset/")

# пробегаем по каждой модели
for model_name in models:
    # определяем устройство, на котором будет выполняться вычисление
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"using {device}")
     # Создаем модель, используя timm и загружаем предварительно обученные веса
    model = timm.create_model(model_name, pretrained=True)
    # Переводим модель в режим оценки
    model.eval()
    # Перемещаем модель на устройство (GPU или CPU), на котором происходят вычисления
    model.to(device)
    # инициализируем счетчики
    tp = 0 # количество правильно угаданных классов
    tp_5 = 0 # количество правильно угаданных классов с учетом пяти наиболее вероятных классов
    _time = 0 # время работы модели
    
    # пробегаем по каждой папке с изображениями
    for folder_name in tqdm(folders):    
        files = os.listdir("./dataset/"+folder_name)
        # пробегаем по каждому изображению в папке
        for file_name in files:
            img = read_img_file("./dataset/"+folder_name+"/"+file_name)
            with torch.no_grad(): # Применяем преобразования к изображению
                img = _transform(img)
                img = torch.unsqueeze(img, 0)
                img = img.to(device)
                # запускаем модель на изображении
                start = timer() # Засекаем время для измерения скорости работы модели
                x = model(img) # Прогоняем изображение через модель и получаем предсказания
                end = timer() # Останавливаем замер времени
                _time+=(end - start) # Добавляем время работы модели к общему времени
                # Применяем функцию softmax для получения вероятностей предсказаний
                x = torch.nn.functional.softmax(x, dim=1) * 100 
                top_5 = torch.topk(x, 5)  # Получаем 5 наиболее вероятных предсказани
            preds = []
            # Получаем метки классов для каждого из наиболее вероятных предсказаний
            for x in top_5.indices[0]:
                preds.append(classes[int(x)])
             # Проверяем, есть ли метка класса папки среди наиболее вероятных предсказаний
            if folder_name in preds[0]:
                tp+=1
           # Проверяем, есть ли метка класса папки среди наиболее вероятных предсказаний в топ-5     
            for pred in preds:
                if folder_name in pred:
                    tp_5+=1
                    break
    # Выводим точность предсказания и время работы модели
    print(f" acc5 = {tp_5/50} acc1 = {tp/50}")
    print(f"time: {_time}")

using cpu


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:28<00:00,  2.84s/it]


 acc5 = 0.84 acc1 = 0.6
time: 8.761995400003798
using cpu


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:37<00:00,  3.77s/it]


 acc5 = 0.88 acc1 = 0.6
time: 17.799644600008833
using cpu


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:25<00:00,  2.58s/it]

 acc5 = 0.9 acc1 = 0.58
time: 6.468407499998648
